This function will check if there's something in front of Thymio. If there's, it will return TRUE and take the control of the Thymio. If there's nothing, it will return FALSE and let the control to optimal path

In [ ]:
def check_cars(state==1)